In [103]:
import pandas as pd
import numpy as np
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())
import requests
import json
from pandas import json_normalize
from sklearn.cluster import KMeans
import folium # map rendering library
import matplotlib.cm as cm
import matplotlib.colors as colors

Scrape the Page http://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [105]:
data = pd.read_html("http://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

Pick the first one with the table we want

In [106]:
df = data[0]

Change the colun name

In [107]:
df.columns = ["PostalCode","Borough","Neighborhood"]

Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [108]:
df = df[df.Borough != 'Not assigned']

In [109]:
coords = pd.read_csv("http://cocl.us/Geospatial_data")

In [110]:
coords.columns = ['PostalCode','Latitude','Longitude']

In [111]:
df2 = pysqldf("select a.*,b.Latitude,b.Longitude from df a join coords b on a.PostalCode = b.PostalCode")

Explore and cluster the neighborhoods in Toronto

In [112]:
#Foursquare Login
CLIENT_ID = 'RIIUOJDODIH3ASACMQJCHEZYD03YNW2JYJPTRRXK1WXWWPNY' # your Foursquare ID
CLIENT_SECRET = 'N5MSSPSUPS1UHNGINLE3JV5CT1CA3M5HIDVMACBF1DRVKI15' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [113]:
#Function that creates the venue list by neighborhood
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [115]:
toronto_venues = getNearbyVenues(names=df2['Neighborhood'],
                                   latitudes=df2['Latitude'],
                                   longitudes=df2['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park / Harbourfront
Lawrence Manor / Lawrence Heights
Queen's Park / Ontario Provincial Government
Islington Avenue
Malvern / Rouge
Don Mills
Parkview Hill / Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park / Princess Gardens / Martin Grove / Islington / Cloverdale
Rouge Hill / Port Union / Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate / Bloordale Gardens / Old Burnhamthorpe / Markland Wood
Guildwood / Morningside / West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor / Wilson Heights / Downsview North
Thorncliffe Park
Richmond / Adelaide / King
Dufferin / Dovercourt Village
Scarborough Village
Fairview / Henry Farm / Oriole
Northwood Park / York University
East Toronto
Harbourfront East / Union Station / Toronto Islands
Little Portugal / Trinity
Kennedy Park / Ionview / East Birchmount Park
Bayview Village
Do

In [117]:
##remove neighborhood as venue
toronto_venues = toronto_venues.loc[toronto_venues['Venue Category'] != 'Neighborhood']

In [118]:
# analyze the neighborhood

# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues['Venue Category'], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']
# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,...,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [119]:
#get the frequency of each catergory by neighborhood
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,...,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo
0,Agincourt,0.000000,0.000000,0.000000,0.020000,0.00,0.00,0.000000,0.000000,0.00,...,0.000000,0.020000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
1,Alderwood / Long Branch,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
2,Bathurst Manor / Wilson Heights / Downsview North,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,...,0.032258,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
3,Bayview Village,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
4,Bedford Park / Lawrence Manor East,0.000000,0.000000,0.000000,0.024390,0.00,0.00,0.000000,0.000000,0.00,...,0.024390,0.000000,0.000000,0.00,0.000000,0.000000,0.024390,0.000000,0.000000,0.00
5,Berczy Park,0.000000,0.000000,0.000000,0.020202,0.00,0.00,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
6,Birch Cliff / Cliffside West,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
7,Brockton / Parkdale Village / Exhibition Place,0.010101,0.000000,0.000000,0.010101,0.00,0.00,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
8,Business reply mail Processing CentrE,0.000000,0.000000,0.000000,0.020833,0.00,0.00,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
9,CN Tower / King and Spadina / Railway Lands / ...,0.000000,0.000000,0.066667,0.000000,0.00,0.00,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00


In [120]:
#define the function that generates top x most category
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [121]:
#
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Chinese Restaurant,Shopping Mall,Caribbean Restaurant,Pizza Place,Bakery,Coffee Shop,Sandwich Place,Supermarket,Sri Lankan Restaurant,Latin American Restaurant
1,Alderwood / Long Branch,Discount Store,Pharmacy,Pizza Place,Convenience Store,Coffee Shop,Sandwich Place,Garden Center,Gas Station,Donut Shop,Liquor Store
2,Bathurst Manor / Wilson Heights / Downsview North,Convenience Store,Coffee Shop,Bank,Pizza Place,Community Center,Ski Area,Frozen Yogurt Shop,Fried Chicken Joint,Sushi Restaurant,Supermarket
3,Bayview Village,Grocery Store,Bank,Gas Station,Japanese Restaurant,Chinese Restaurant,Park,Shopping Mall,Restaurant,Café,Trail
4,Bedford Park / Lawrence Manor East,Italian Restaurant,Coffee Shop,Bank,Restaurant,Sandwich Place,Fast Food Restaurant,Bridal Shop,Skating Rink,Intersection,Juice Bar


In [122]:
latitude = toronto_merged.Latitude.mean()

In [123]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)


In [124]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = pysqldf("select a.*,b.Latitude,b.Longitude from neighborhoods_venues_sorted a join df2 b on a.Neighborhood = b.Neighborhood")

toronto_merged.head() # check the last columns!

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Latitude,Longitude
0,4,Agincourt,Chinese Restaurant,Shopping Mall,Caribbean Restaurant,Pizza Place,Bakery,Coffee Shop,Sandwich Place,Supermarket,Sri Lankan Restaurant,Latin American Restaurant,43.794200,-79.262029
1,4,Alderwood / Long Branch,Discount Store,Pharmacy,Pizza Place,Convenience Store,Coffee Shop,Sandwich Place,Garden Center,Gas Station,Donut Shop,Liquor Store,43.602414,-79.543484
2,4,Bathurst Manor / Wilson Heights / Downsview North,Convenience Store,Coffee Shop,Bank,Pizza Place,Community Center,Ski Area,Frozen Yogurt Shop,Fried Chicken Joint,Sushi Restaurant,Supermarket,43.754328,-79.442259
3,4,Bayview Village,Grocery Store,Bank,Gas Station,Japanese Restaurant,Chinese Restaurant,Park,Shopping Mall,Restaurant,Café,Trail,43.786947,-79.385975
4,0,Bedford Park / Lawrence Manor East,Italian Restaurant,Coffee Shop,Bank,Restaurant,Sandwich Place,Fast Food Restaurant,Bridal Shop,Skating Rink,Intersection,Juice Bar,43.733283,-79.419750


In [125]:
# create map
latitude = toronto_merged.Latitude.mean()
longitude = toronto_merged.Longitude.mean()
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters